In [119]:
from qiskit import QuantumCircuit
from qiskit_aer.primitives import Estimator
import matplotlib.pyplot as plt
import numpy as np
from qiskit.quantum_info import Statevector, random_clifford

from tomography import Random_Measurements
from tomography import NearSparseTomography 

In [120]:
NQs = [ 2, 3, 4, 5, 6 ]
shots = 1000
simulator_ideal=Estimator(backend_options={'shots':shots,
                                            'method':"stabilizer",},
                            transpile_options={'optimization_level':0},
                            abelian_grouping=True, ) 

In [126]:
MC = 10
Fids = np.zeros( [len(NQs),MC] )
num_meas = [15, 40, 100, 200, 350]

for i, NQ in enumerate(NQs):
    
    d = 2**NQ

    for j in range(MC):
        RM = Random_Measurements(NQ)

        psi_circ = random_clifford( NQ ).to_circuit()
        psi_th = np.array( Statevector(psi_circ) )

        RM.RandomMeasurements( num_meas[i], psi_circ, simulator_ideal )

        phi_in = np.random.rand(d) + 1j * np.random.rand(d)
        phi_in = phi_in / np.linalg.norm(phi_in)

        psi_out = NearSparseTomography( phi_in, RM )

        fid = np.abs( np.vdot( psi_th, psi_out ) )**2 
        
        Fids[i,j] = fid

In [127]:
Fids.round(2)

array([[1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [0.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ],
       [0.99, 1.  , 0.  , 1.  , 0.  , 0.  , 1.  , 0.99, 1.  , 1.  ],
       [0.99, 0.04, 0.  , 0.25, 0.99, 0.98, 0.  , 0.  , 0.  , 0.  ]])

In [129]:
np.mean( Fids, axis=1 )

array([0.99944591, 0.99877981, 0.89788466, 0.69687072, 0.32502589])